In [ ]:
%cd ~
import pyfits,glob,time,scipy, sys
from scipy.special import erf
from pylab import *
from subprocess import call
from scipy.ndimage import gaussian_filter, convolve

topdir='/Users/cslage/Research/LSST/code/'
#thedir=topdir+'brighter_fatter/20170831_002_spots/'
thedir=topdir+'GUI/brighter_fatter/20171128_002_spots_VBB60/'
kerneldir = topdir+'poisson/Poisson_CCD_Hole20_Test11E/'
%cd $thedir

configfile=topdir+'sextractor/default-array_dither.sex'
paramfile=topdir+'sextractor/default-array_dither.param'
maskdir=topdir+'sextractor/masks/'



In [ ]:
zfilelist=sort(glob.glob(thedir+'ITL-3800C-002_spot_spot_450_20171128??????.fits'))
print len(zfilelist)

#### First get some spot data

In [ ]:
def remove_overscan_xy(image,x_overscan_start,x_overscan_end,y_overscan_start,y_overscan_end):
    overscan=image[:y_overscan_start,x_overscan_start+1:x_overscan_end]
    image=image[:y_overscan_start,:x_overscan_start]
    finalimg=image-matrix(median(overscan,axis=1)).T*np.ones(shape(image)[1])
    return array(finalimg)

class Array2dSet:
    def __init__(self,xmin,xmax,nx,ymin,ymax,ny,nstamps):
        # This packages up a set of nstamps postage stamp images,
        # each image of which is nx * ny pixels
        self.nx=nx
        self.ny=ny
        self.nstamps=nstamps

        self.xmin=xmin
        self.ymin=ymin
        
        self.xmax=xmax
        self.ymax=ymax
        
        self.dx=(xmax-xmin)/nx
        self.dy=(ymax-ymin)/ny
        
        self.x=linspace(xmin+self.dx/2,xmax-self.dx/2,nx)
        self.y=linspace(ymin+self.dy/2,ymax-self.dy/2,ny)

        self.data=zeros([nx,ny,nstamps])
        self.xoffset=zeros([nstamps])
        self.yoffset=zeros([nstamps])
        self.imax=zeros([nstamps])


def BuildSpotList(fitsfilename, segmentnumber, numspots, nx, ny, minsize, maxsize):
    stampxmin = -(int(nx/2)+0.5)
    stampxmax = -stampxmin
    stampymin = -(int(ny/2)+0.5)
    stampymax = -stampymin
    xcoomin = 50
    xcoomax = 250
    ycoomin = 1500
    ycoomax = 1700
    spotlist = Array2dSet(stampxmin,stampxmax,nx,stampymin,stampymax,ny,numspots)
    hdr=pyfits.getheader(fitsfilename,segmentnumber)
    extname = hdr['EXTNAME']
    img=pyfits.getdata(fitsfilename,extname) 
    overscansubimg=remove_overscan_xy(img,509,542,2000,2022) 
    catname=fitsfilename[:-5]+extname+'.fits.cat.reg' 
    catfile = open(catname,'r')
    catlines = catfile.readlines()
    catfile.close()
    n=0
    for line in catlines:
        try:
            size = float(line.split()[3].split('#')[0])
            if size < minsize or size > maxsize:
                continue
            xcoord = float(line.split()[1])
            ycoord = float(line.split()[2])
            if xcoord < xcoomin or xcoord > xcoomax or ycoord < ycoomin or ycoord > ycoomax:
                continue
            xint = int(xcoord-0.5)
            yint = int(ycoord-0.5)
            xmin = xint - int(nx/2)
            xmax = xint + int(nx/2) + 1
            ymin = yint - int(ny/2)
            ymax = yint + int(ny/2) + 1
            stamp = overscansubimg[ymin:ymax, xmin:xmax]
           
            xsum = 0.0
            ysum = 0.0
            datasum = 0.0
             
            for i in range(nx):
                for j in range(ny):
                    spotlist.data[i,j,n] = float(stamp[j,i])                    
                    ysum += spotlist.y[j] * spotlist.data[i,j,n]
                    xsum += spotlist.x[i] * spotlist.data[i,j,n]
                    datasum += spotlist.data[i,j,n]
            xoff = xsum / datasum
            yoff = ysum / datasum
            spotlist.xoffset[n] = xoff
            spotlist.yoffset[n] = yoff
            #spotlist.xoffset[n] = xcoord - xint - 1.0
            #spotlist.yoffset[n] = ycoord - yint - 1.0     
            #print "Spot = %d, calc_off = (%f,%f), sex_off = (%f,%f)"%(n,xoff,yoff,xcoord-xint-1.0,ycoord-yint-1.0)
            n += 1
            if n == numspots:
                return spotlist
        except:
            continue
    # Reaching this point means we found less spots than requested.
    newspotlist = Array2dSet(stampxmin,stampxmax,nx,stampymin,stampymax,ny,n)
    newspotlist.xoffset = spotlist.xoffset[0:n]
    newspotlist.yoffset = spotlist.yoffset[0:n]
    newspotlist.data = spotlist.data[:,:,0:n]
    del spotlist
    return newspotlist


def Area(xl, xh, yl, yh, sigmax, sigmay, Imax):
    # Calculates how much of a 2D Gaussian falls within a rectangular box
    ssigx = sqrt(2) * sigmax
    ssigy = sqrt(2) * sigmay    
    I = (erf(xh/ssigx)-erf(xl/ssigx))*(erf(yh/ssigy)-erf(yl/ssigy))
    return Imax * I / 4.0


def FOM(params):
    [sigmax, sigmay] = params
    result = forward.forward(spotlist,sigmax,sigmay)
    return result

def PyFOM(params):
    fom = 0.0
    [Imax, sigmax, sigmay] = params
    area=zeros([spotlist.nx,spotlist.ny])
   
    for spot in range(spotlist.nstamps):
        for ii in range(spotlist.nx):
            for jj in range(spotlist.ny):
                xl = spotlist.x[ii] - spotlist.xoffset[spot] - 0.5
                xh = xl + 1.0
                yl = spotlist.y[jj] - spotlist.yoffset[spot] - 0.5
                yh = yl + 1.0
                #if spot == 78 and ii == 4 and jj == 4 or spot==0:
                    #print "ii = %d, jj = %d, img = %.4f"%(ii,jj,spotlist.data[ii,jj,spot])
                #print "ii = %d, jj = %d,xl = %.2f, xh = %.2f, yl = %.2f, yh = %.2f"%(ii,jj,xl,xh,yl,yh)
                area[ii,jj] = AreaG(xl, xh, yl, yh, sigmax, sigmay, Imax)
                fom += square(area[ii,jj]-spotlist.data[ii,jj,spot])
    #print "Imax = %.1f, sigmax = %.2f, sigmay = %.2f, fom = %.1f"%(Imax, sigmax, sigmay, fom)
    return fom


In [ ]:
nx = 11
ny = 11
numspots =10
spotnum = 7
segmentnumber = 13

fitsfilename = zfilelist[0]
print fitsfilename
    
spotlist = BuildSpotList(fitsfilename, segmentnumber, numspots, nx, ny, 0.7, 1.4)
print "nstamps = %d"%spotlist.nstamps
spot = 4.0 * spotlist.data[:,:,spotnum] # Convert to electrons
print "xoff = %f, yoff = %f"%(spotlist.xoffset[spotnum],spotlist.yoffset[spotnum])

imshow(spot, interpolation='nearest')
colorbar()
show()

#### Now extract the kernel

In [ ]:
class Array2d:
    def __init__(self,xmin,xmax,nx,ymin,ymax,ny):
        # Each image is nx * ny pixels
        self.nx=nx
        self.ny=ny

        self.xmin=xmin
        self.ymin=ymin
        
        self.xmax=xmax
        self.ymax=ymax
        
        self.dx=(xmax-xmin)/nx
        self.dy=(ymax-ymin)/ny
        
        self.x=linspace(xmin+self.dx/2,xmax-self.dx/2,nx)
        self.y=linspace(ymin+self.dy/2,ymax-self.dy/2,ny)

        self.cov=zeros([nx,ny])
        self.kernel=zeros([nx,ny])        

def ReadConfigFile(filename):
    # This reads the Poisson simulator config file for
    # the settings that were run
    # and returns a dictionary with the values

    with open(filename,'r') as file:
        lines=file.readlines()
    lines = [ l.strip() for l in lines ]
    lines = [ l.split() for l in lines if len(l) > 0 and l[0] != '#' ]
    for line in lines:
        if line[1] != '=':
            print "Check line: ",line
            raise IOError("Error reading config file %s"%filename)
    config = {}
    for line in lines:
        try:
            # First get the ordered pairs
            config.update({line[0]:[eval(line[2]), eval(line[3])]})
        except:
            try:
                # Then get the numeric values
                config.update({line[0]:eval(line[2])})
            except:
                try:
                    # Last, get the strings
                    config.update({line[0]:str(line[2])})
                except:
                    pass
    return config


def ReadAreaFile(filename, arr, Area_0, NxCenter, NyCenter):
    # This reads the simulated pixel area file
    # and returns an array with the covariances
    file = open(filename, 'r')
    lines = file.readlines()
    file.close()
    lines.remove(lines[0]) # Strip the title line    
    for line in lines:
        items = line.split()
        i = int(items[0])
        j = int(items[1])
        area = float(items[2])
        ii = i + (arr.nx - 1)/2 - NxCenter 
        jj = j + (arr.ny - 1)/2 - NyCenter 
        arr.cov[ii,jj] += (area - Area_0) / Area_0
    return arr

def ReadMeasurementFile(filename, arr):
    # This reads the measured correlation data file
    # and returns an array with the covariances
    file = open(filename, 'r')
    lines = file.readlines()
    file.close()
    lines.remove(lines[0]) # Strip the title line
    for i in range(3):
        lines.remove(lines[-1]) # Strip the last three lines

    for line in lines:
        items = line.split()
        i = int(items[0])
        j = int(items[1])
        cov = float(items[2])
        if (i > 0 or j > 0) and cov < 0.0:
            cov = 0.0
        for xsign in [-1,1]:
            ii = i * xsign + (arr.nx - 1)/2
            for ysign in [-1,1]:
                jj = j * ysign + (arr.ny - 1)/2
                arr.cov[jj,ii] = cov
    return arr


def SOR(arr, w):
    hsquared =  arr.dx * arr.dy;
    omw = 1.0 - w;
    w4 = w / 4.0
    for i in range(1, arr.nx-1):
        for j in range(1, arr.ny-1):
	    arr.kernel[i,j] = omw * arr.kernel[i,j] + w4 * (arr.kernel[i-1,j] + arr.kernel[i+1,j] + arr.kernel[i,j-1] + arr.kernel[i,j+1] + hsquared * arr.cov[i,j])
    return arr
  

In [ ]:
# First, read the .cfg file
%cd $kerneldir
configfile = kerneldir+'transdata/pixel2/pixel.cfg'
run = 0
ConfigData = ReadConfigFile(configfile)
outputfilebase = ConfigData["outputfilebase"]
outputfiledir = ConfigData["outputfiledir"]
Nx = ConfigData["PixelBoundaryNx"]
Ny = ConfigData["PixelBoundaryNy"]
XCenter = ConfigData["FilledPixelCoords_0_0"][0]
YCenter = ConfigData["FilledPixelCoords_0_0"][1]
PixelSizeX = ConfigData["PixelSizeX"]
PixelSizeY = ConfigData["PixelSizeY"]
NxCenter = int((XCenter - ConfigData["PixelBoundaryLowerLeft"][0]) / PixelSizeX)
NyCenter = int((YCenter - ConfigData["PixelBoundaryLowerLeft"][1]) / PixelSizeY)
Area_0 = 99.994058# Nominally 100.0, running with no charge gives 99.99745. this value minimizes sum(cov)
NumElec = ConfigData["CollectedCharge_0_0"]
NewNx = NewNy = 21
NewNxCenter = (NewNx - 1) / 2
NewNyCenter = (NewNy - 1) / 2

filename = outputfiledir + '/' + outputfilebase +'_%d_Area'%run + '.dat'
kernel = Array2d(0,NewNx,NewNx,0,NewNy,NewNy)
kernel = ReadAreaFile(filename, kernel, Area_0, NxCenter, NyCenter)
kernel.cov /= NumElec

w = 1.9
for n in range(1000):
    kernel = SOR(kernel,w)

imshow(kernel.kernel, interpolation='nearest')
show()

In [ ]:
# What does the convolution of the kernel with the spot look like?
spot_kernel=convolve(spot,kernel.kernel,mode='constant',cval=0.0)
imshow(spot_kernel, interpolation='nearest')
colorbar()
show()

In [ ]:
# Now implement the correction
def Dx(image):
    dx = zeros(image.shape)
    for i in range(1,image.shape[0]-1):
        for j in range(image.shape[1]):
            dx[i,j] = (image[i+1,j] - image[i-1,j])/2.0
    return dx

def Dy(image):
    dy = zeros(image.shape)
    for i in range(image.shape[0]):
        for j in range(1,image.shape[1]-1):
            dy[i,j] = (image[i,j+1] - image[i,j-1])/2.0
    return dy

def D2(image):
    d2 = zeros(image.shape)
    for i in range(1,image.shape[0]-1):
        for j in range(1,image.shape[1]-1):
            d2[i,j] = (image[i+1,j] + image[i-1,j] + image[i,j+1] + image[i,j-1] - 4.0 * image[i,j])
    return d2

def Correction(data, kernel):
    int_term = convolve(data,kernel,mode='constant',cval=0.0)
    term1 = Dx(data) * Dx(int_term) + Dy(data)* Dy(int_term)
    term2 = data * D2(int_term)
    correction = (term1 + term2) / 2.0
    return correction

correction = Correction(spot, kernel.kernel)
imshow(correction, interpolation ='nearest')
colorbar()
show()

In [ ]:
newspot = copy(spot)
correction = zeros(spot.shape)
diff = 100.0
# Iterate to convergence
while diff > 1.0E-6:
    lastcorrection = copy(correction)
    correction = Correction(newspot, kernel.kernel)
    newspot = spot + correction
    diff = ((lastcorrection - correction) * (lastcorrection - correction)).sum()
    print "Diff = %f"%diff

print correction.max(), correction.min()
figure(figsize=(16,8))
subplots_adjust(wspace=0.5)
subplot(2,3,1)
title("X")
plot(spotlist.x, spot[:,5], label='Spot')
plot(spotlist.x, newspot[:,5], label='NewSpot')
plot(spotlist.x, 20*correction[:,5], label='20X Correction')
subplot(2,3,2)
title("Y")
plot(spotlist.y, spot[5,:], label='Spot')
plot(spotlist.y, newspot[5,:], label='NewSpot')
plot(spotlist.y, 20*correction[5,:], label='20X Correction')
legend(loc='center right', bbox_to_anchor=(2.0, 0.5))
subplot(2,3,4)
title("Spot")
imshow(spot, interpolation = 'nearest')
subplot(2,3,5)
title("NewSpot")
imshow(newspot, interpolation = 'nearest')
subplot(2,3,6)
title("Correction")
imshow(correction, interpolation = 'nearest')
colorbar()
#show()
savefig(thedir+"Coulton_Correction_7May18.png")